
# 🌐 Open‑Source LLM Landscape (2025)

**Prompt Engineering — Comprehensive Colab Notebook**

---

### Learning Objectives
1. **Survey** the major open‑source language‑model families, sizes, and licenses.  
2. **Load & run** a lightweight OSS model via 🤗 Transformers.  
3. **Compare** performance on the Open‑LLM Leaderboard.  
4. **Understand** common permissive vs. restricted licenses.  
5. **Explore** quantization & `llama.cpp` for local inference.  
6. **Preview** evaluation & fine‑tuning workflows (LoRA/QLoRA).  



## ⏳ Table of Contents
1. [Introduction](#intro)  
2. [Setup & Dependencies](#setup)  
3. [First Hands‑On Inference](#hands-on)  
4. [Model Family Overview](#families)  
5. [Licensing 101](#license)  
6. [Quantization & `llama.cpp`](#quant)  
7. [Evaluation Quick‑Start](#eval)  
8. [Fine‑Tuning Preview](#finetune)  
9. [Exercises](#ex)  
10. [Further Reading](#read)  



<a id='intro'></a>
## 1️⃣ Introduction — Why Open Source LLMs?

Open‑source language models accelerated **research reproducibility**, **deployment flexibility**, and a **lower barrier to entry** for startups, hobbyists, and academia. While proprietary frontier models often dominate raw benchmark scores, OSS models:

* enable full‑stack auditing & transparency  
* foster community‑driven safety improvements  
* allow on‑prem or air‑gapped deployments  
* spur innovation through forks and fine‑tunes  


<a id='setup'></a>
## 2️⃣ Setup & Dependencies

In [ ]:
# ↳ This installs lightweight baseline deps (GPU‑ready on Colab)
!pip -q install --upgrade transformers accelerate sentencepiece bitsandbytes --progress-bar off


<a id='hands-on'></a>
## 3️⃣ Hands‑On — Load and Chat with **TinyLlama‑1.1B‑Chat**

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch, textwrap

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
print(f"Loading {model_id}...")

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

def chat(prompt, max_new_tokens=128, temperature=0.7):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs,
                                 max_new_tokens=max_new_tokens,
                                 temperature=temperature,
                                 do_sample=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print(chat("Explain in simple terms what makes open‑source language models important.", 64))


> ⚠️ *Colab’s free tier may OOM for models >4 B parameters.*  
Try switching to a T4/RTX‑A100 runtime, or use a GGUF quantized model via `llama.cpp` (see Section 6).

<a id='families'></a>
## 4️⃣ Model Family Overview — Sizes, Licenses, Use‑Cases

In [ ]:
import pandas as pd

models = [
    ("Llama‑3‑8B‑Instruct", "Meta", "8 B", "LLAMA‑3 Community", "2025‑04", "General chat, coding"),
    ("Mixtral‑8×22B", "Mistral AI", "176 B (MoE)", "Apache‑2.0", "2024‑12", "High‑performance chat"),
    ("Qwen1.5‑7B‑Chat", "Alibaba (Qwen)", "7 B", "Qianwen License v2", "2025‑01", "Multilingual, instruction"),
    ("Gemma‑7B‑It", "Google DeepMind", "7 B", "Gemma License", "2024‑02", "Research, fine‑tuning"),
    ("Phi‑3‑mini‑4k‑instruct", "Microsoft", "3.8 B", "MIT", "2025‑05", "Reasoning, mobile"),
    ("TinyLlama‑1.1B‑Chat", "TinyLlama", "1.1 B", "Apache‑2.0", "2024‑10", "Lightweight demos")
]

df = pd.DataFrame(models, columns=["Model", "Org", "Params", "License", "Release", "Notes"])
df



<a id='license'></a>
## 5️⃣ Licensing 101 — Permissive vs. Restricted

| License | Permissive? | Commercial? | Derivatives? |
|---------|-------------|-------------|--------------|
| **Apache‑2.0** | ✅ | ✅ | ✅ w/ notice |
| **MIT** | ✅ | ✅ | ✅ w/ notice |
| **LLAMA 3 Community** | ⚠️ up to 700 M MAU | Limited | ✅ but cannot compete |
| **Qianwen v2** | ⚠️ | Case‑by‑case | Same license |
| **Gemma License** | ⚠️ | Free research; conditional commercial | Attribution & policy compliance |

> Always read the full text! Even “open” licenses may cap monthly active users or restrict model competition.



<a id='quant'></a>
## 6️⃣ Quantization & `llama.cpp`

`llama.cpp` lets you run GGUF‑quantized models (e.g., 4‑bit) on CPU‑only systems.

```bash
# ≈ 40‑second build on Colab
apt-get -qq install -y build-essential cmake
pip install --quiet llama-cpp-python

# Download a 4‑bit TinyLlama GGUF (≈ 500 MB)
wget -q https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-GGUF/resolve/main/tinyllama-1.1b-chat.q4_K_M.gguf -O tiny.gguf
```

```python
from llama_cpp import Llama
llm = Llama(model_path="tiny.gguf", n_ctx=2048)
print(llm("Q: What is the capital of France?\nA:", max_tokens=20)["choices"][0]["text"])
```



<a id='eval'></a>
## 7️⃣ Evaluation Quick‑Start

```bash
pip -q install lm-eval==0.4.2
lm-eval --model hf --model_args pretrained=TinyLlama/TinyLlama-1.1B-Chat-v1.0 --tasks truthful_qa --batch_size 8
```

This runs the **TruthfulQA** benchmark and outputs an accuracy percentage that you can compare with the [Open‑LLM Leaderboard](https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard).



<a id='finetune'></a>
## 8️⃣ Fine‑Tuning Preview — LoRA in ~60 Lines

```python
!pip -q install peft datasets

from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id, torch_dtype=torch.float16, device_map="auto"
)

peft_cfg = LoraConfig(task_type="CAUSAL_LM", r=8, lora_alpha=32, lora_dropout=0.05)
model = get_peft_model(model, peft_cfg)

dataset = load_dataset("Abirate/english_quotes", split="train[:1%]").map(
    lambda x: {"text": x["quote"]}
)

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=64)
dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])

training_args = TrainingArguments(
    output_dir="tinyllama-lora",
    per_device_train_batch_size=8,
    num_train_epochs=1,
    fp16=True,
    logging_steps=20,
    save_steps=1000,
)

trainer = Trainer(model, training_args, train_dataset=dataset)
trainer.train()
```

This fine‑tunes only **3 %** of TinyLlama’s parameters thanks to LoRA adapters.



<a id='ex'></a>
## 9️⃣ Exercises
1. **Swap Models:** Replace TinyLlama with `phi-3-mini-4k-instruct` and re‑run Section 3.  
2. **License Audit:** Using the overview table, mark which models you could ship in a SaaS with >1 M MAU.  
3. **Evaluation Challenge:** Run **HellaSwag** on two models and compare scores.  
4. **LoRA Sprint:** Fine‑tune Gemma‑2B‑It on 100 lines of your own chat data.  



<a id='read'></a>
## 🔗 Further Reading & Leaderboards
* Mistral AI — “Cheaper, Better, Faster, Stronger” release blog  
* Meta — LLAMA 3 license and paper  
* Alibaba Cloud — Qwen 1.5 Research License  
* Hugging Face **Open‑LLM Leaderboard**  
* *“Sparse Mixture‑of‑Experts Are All You Need”* (Mistral 2024)  
* *“QLoRA: Efficient Fine‑Tuning of Quantized LLMs”* (ICML 2024)  

Happy prompting! 🚀
